In [ ]:
import pandas as pd
import numpy as np

In [ ]:
entities = pd.read_csv('../ckpts/ComplEx_triple_1/entities.tsv',delimiter='\t',names = ['id','entity'])
relations = pd.read_csv('../ckpts/ComplEx_triple_1/relations.tsv',delimiter='\t',names = ['id','rel'])

In [ ]:
entities[:5]

In [ ]:
entity2id = {}
id2entity = {}
for i in range(entities.shape[0]):
    entity2id[entities['entity'].values[i]] = i
    id2entity[i] = entities['entity'].values[i]

In [ ]:
rel2id = {}
id2rel = {}
for i in range(relations.shape[0]):
    rel2id[relations['rel'].values[i]] = i
    id2rel[i] = relations['rel'].values[i]

In [ ]:
rel_emb = np.load('../ckpts/ComplEx_triple_1/triple_ComplEx_relation.npy')
print(rel_emb.shape)
entity_emb = np.load('../ckpts/ComplEx_triple_1/triple_ComplEx_entity.npy')
print(entity_emb.shape)

In [ ]:
# relation_list = [9, 32,37, 46, 51, 7, 50]
relation_list = [37]
with open('./rel.list', 'w') as f:
    for item in relation_list:
        print(item)
        f.write("%s\n" % item)

In [ ]:
head = entities[entities.entity.str.contains('Compound') & (~entities.entity.str.contains('MESH')) &
                (~entities.entity.str.contains('CHEBI'))]
head_list = head.id.values.tolist()
# head_list = [int(head.id.iloc[0])]

# head_name = 'Compound::DB08907' #Canagliflozin
# entity2id[head_name]
# head_list = [145]
with open('head.list', 'w') as f:
    for item in head_list:
        f.write("%s\n" % item)

#### using dglke_precit command for link prediction.

In [ ]:
!DGLBACKEND=pytorch dglke_predict --model_path ckpts/ComplEx_triple_1/ --format 'h_r_*' --data_files head.list rel.list --score_func none --topK 100000 --gpu 0

In [ ]:
triplets_df = pd.read_csv('../triplets_final.tsv', sep = '\t', names = ['h','r','t'])
result_df = pd.read_csv('./result.tsv', sep = '\t')

In [ ]:
#head = 'Compound::DB09292' # Drug: Sacubitril (Entresto)
#head = 'Compound::DB00177' # Drug Valsartan (Entresto)
head = 'Compound::DB08868' # Drug Fingolimod (Gilenya)

triplets = triplets_df[triplets_df.h.str.contains(head) & triplets_df.r.str.contains('Compound:Disease')] 

entity2id[head]
results = result_df[result_df['head'] == entity2id[head]]
results['tail1'] = [id2entity[results['tail'].iloc[id1]] for id1 in range(results.shape[0])]
results['predict'] = results['tail1'].isin(triplets.t)

predict_results = results[results['tail1'].str.contains('Disease') & (~results['tail1'].isin(triplets.t))]
for id1 in range(predict_results.shape[0]):
    print(id2entity[predict_results['tail'].iloc[id1]])

In [ ]:
predict_results.head(19)